In [8]:

import pandas as pd
import Levenshtein as lv
import numpy as np

def prePro(df):
    ls = []
    for i in range(df.shape[0]):
        temp = lv.distance(df.iloc[i,0],df.iloc[i,1])
        ls.append(temp)
    
    df['Levenshtein'] = ls
    tf = df[df.iloc[:,2] == 1]
    
    return tf


In [9]:

def aPos(char):
    n={'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 
                 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 
                 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 
                 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
    
    return n[char]

import pandas as pd
import Levenshtein as lv
import numpy as np


def prePro(df):
    
    df.replace('\'','', regex=True, inplace=True)

    ls = []
    for i in range(df.shape[0]):
        temp = lv.distance(df.iloc[i,0],df.iloc[i,1])
        ls.append(temp)
    
    df['Levenshtein'] = ls
    tf = df[df.iloc[:,2] == 1]
   
    tf = tf.drop(tf.index[276])
    
    return tf
def prior():
    df = pd.read_csv("pp_data.csv")
    df = df.applymap(lambda a: str(a).lower())



mispellings = pd.read_csv("pp_mispellings.csv")
mispellings = prePro(mispellings)
# mispellings = mispellings.iloc[:3,:]



ins = np.zeros([27,26], dtype = 'int')
s = np.zeros([26,26], dtype = 'int')
d = np.zeros([27,26], dtype = 'int')



for i in range(mispellings.shape[0]):
    correct = mispellings.iloc[i,0]  #1
    wrong = mispellings.iloc[i,1] #0
    seq = lv.editops(correct,wrong)
    for j in seq:
        if j[0] == "insert": #need null character for these
            if j[1] == 0:
                p1 = 26 
            else:
                p1 = aPos(correct[j[1]-1])
            p2 = aPos(wrong[j[2]])
            ins[p1,p2] = ins[p1,p2] + 1
            
        elif j[0] == "delete": #need null character for these
            # print(correct,wrong,j[1],j[2])
            if j[1] == 0:
                p1 = 26 
            else:
                p1 = aPos(correct[j[1]-1]) 
            p2 = aPos(correct[j[1]])
            # print("DEL",correct,wrong,p1,p2)
            d[p1,p2] = d[p1,p2] + 1
        else: # j[0] == "replace":
            p1 = aPos(correct[j[1]]); p2 = aPos(wrong[j[2]])
            s[p1,p2] = s[p1,p2] + 1
        


In [10]:
totalerr=sum(sum(ins)+sum(d)+sum(s))
totalerr

1465

In [11]:
vocab=pd.read_csv("pp_data.csv")

In [12]:
vocab

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
70645,defoliate,12718
70646,insatiably,12717
70647,cestus,12716
70648,rasbora,12714


In [13]:
total=sum(vocab["count"])


In [14]:
dictionary=vocab

x="hillo"
tmp=list()
tmp1=list()
dfa=pd.DataFrame()
for i in range(len(dictionary)):
    if lv.distance(x,str(dictionary.loc[i,"word"]))==1:
        tmp.append(str(dictionary.loc[i,"word"]))
        tmp1.append(dictionary.loc[i,"count"])
    else:
        None
tmp2=[i/sum(tmp1) for i in tmp1]
tmp3=[i/total for i in tmp1]  

dfa["correct"]=tmp
dfa["prob word"]=tmp2
dfa["prob word g"]=tmp3

proabb=list()
for g in tmp:
    tmpl=lv.editops(x,g)[0]
    if tmpl[0]=="delete":
        tmpl1=aPos(x[int(tmpl[1])])
        tmpl2=aPos(x[int(tmpl[1]-1)])
        prob=d[tmpl1,tmpl2]/totalerr
        proabb.append(prob)
        
    elif tmpl[0]=="replace":
        tmpl1=aPos(x[int(tmpl[1])])
        tmpl2=aPos(g[int(tmpl[1])])
        prob=s[tmpl1,tmpl2]/totalerr
        proabb.append(prob)
        
    else:
        tmpl1=aPos(x[int(tmpl[1])])
        tmpl2=aPos(x[int(tmpl[1]-1)])
        prob=ins[tmpl1,tmpl2]/totalerr
        proabb.append(prob)


dfa["prob of mispel"]=prob

totalprob1=list()
for w in range(len(dfa)):
    tmppp=dfa.loc[w,"prob word g"]*dfa.loc[w,"prob of mispel"]
    totalprob1.append(tmppp)

dfa["total prob"]=totalprob1
dfa1=dfa
dfa1=dfa1.sort_values(by='total prob', ascending=False)
dfa1

,correct,prob word,prob word g,prob of mispel,total prob
0,hill,0.519337,1.213702e-04,0.000683,8.284658e-08
1,hello,0.267426,6.249820e-05,0.000683,4.266089e-08
2,hills,0.209089,4.886449e-05,0.000683,3.335460e-08
3,hilly,0.004148,9.694409e-07,0.000683,6.617344e-10


In [15]:
dfa

,correct,prob word,prob word g,prob of mispel,total prob
0,hill,0.519337,1.213702e-04,0.000683,8.284658e-08
1,hello,0.267426,6.249820e-05,0.000683,4.266089e-08
2,hills,0.209089,4.886449e-05,0.000683,3.335460e-08
3,hilly,0.004148,9.694409e-07,0.000683,6.617344e-10


In [16]:
vocab.head()

,word,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698


In [17]:
#lv.editops(c,w)
l=lv.editops("hell","hello")[0]
l

('insert', 4, 4)

In [18]:
lv.distance("helo","heello")

2

In [19]:
f="hello"
f[3]

'l'

In [20]:
#Mispelling checker

def misp(x):
    tmp=list()
    tmp1=list()
    dfa=pd.DataFrame()
    for i in range(len(dictionary)):
        if lv.distance(x,str(dictionary.loc[i,"word"]))==1:
            tmp.append(str(dictionary.loc[i,"word"]))
            tmp1.append(dictionary.loc[i,"count"])
        else:
            None
    tmp2=[i/sum(tmp1) for i in tmp1]
    tmp3=[i/total for i in tmp1]  

    dfa["correct"]=tmp
    dfa["prob word"]=tmp2
    dfa["prob word g"]=tmp3

    proabb=list()
    for g in tmp:
        tmpl=lv.editops(x,g)[0]
        if tmpl[0]=="delete":
            tmpl1=aPos(x[int(tmpl[1])])
            tmpl2=aPos(x[int(tmpl[1]-1)])
            prob=d[tmpl1,tmpl2]/totalerr
            proabb.append(prob)

        elif tmpl[0]=="replace":
            tmpl1=aPos(x[int(tmpl[1])])
            tmpl2=aPos(g[int(tmpl[1])])
            prob=s[tmpl1,tmpl2]/totalerr
            proabb.append(prob)

        else:
            tmpl1=aPos(g[int(tmpl[1])])
            tmpl2=aPos(x[int(tmpl[1]-1)])
            prob=ins[tmpl1,tmpl2]/totalerr
            proabb.append(prob)


    dfa["prob of mispel"]=proabb

    totalprob1=list()
    for w in range(len(dfa)):
        tmppp=dfa.loc[w,"prob word g"]*dfa.loc[w,"prob of mispel"]
        totalprob1.append(tmppp)

    dfa["total prob"]=totalprob1
    dfa1=dfa
    dfa1=dfa1.sort_values(by='total prob', ascending=False)
    return dfa1

In [21]:
misp("accress")

,correct,prob word,prob word g,prob of mispel,total prob
1,actress,0.031156,0.000013,0.000683,9.073172e-09
0,access,0.968844,0.000413,0.000000,0.000000e+00


In [22]:
data= pd.read_csv("mispellings.csv")

In [23]:
data1=data
data1=data1.sort_values(by='lev', ascending=True)
data3=pd.DataFrame()
data3["actual word"]=data1["mispellings"][2:1540]
data3["mispelling"]=data1["actual word"][2:1540]

KeyError: 'lev'

In [ ]:
data4=data3.reset_index()
data4=data4.drop("index", axis=1)

In [ ]:
data4

,actual word,mispelling
0,Apennines,Apenines
1,infiltrated,infilitrated
2,infiltrate,infilitrate
3,infectious,infectuous
4,inevitable,inevitible
...,...,...
1533,conscious,concious
1534,conquerors,conquerers
1535,consonants,consonents
1536,consonant,consonent


In [ ]:
guessed=list()

for g in data4["mispelling"]:
    try:
        tmpdfgt=misp(g)
        if len(tmpdfgt)>0:
            gue=tmpdfgt.loc[0,"correct"]
            guessed.append(gue)
        else:
            guessed.append("Nan")
    except:
        guessed.append("Error")
data4["guess"]=guessed

In [ ]:
s.shape

(26, 26)

In [36]:
misp("hillo")


,correct,prob word,prob word g,prob of mispel,total prob
2,poem,0.058858,1.689280e-05,0.000683,1.153092e-08
4,poon,0.001749,5.018892e-07,0.003413,1.712933e-09
5,peen,0.000373,1.071331e-07,0.015017,1.608825e-09
0,porn,0.788140,2.262025e-04,0.000000,0.000000e+00
1,pen,0.108159,3.104253e-05,0.000000,0.000000e+00
3,poet,0.042721,1.226133e-05,0.000000,0.000000e+00


In [34]:
misp("accress")

,correct,prob word,prob word g,prob of mispel,total prob
1,actress,0.031156,0.000013,0.000683,9.073172e-09
0,access,0.968844,0.000413,0.000000,0.000000e+00


In [24]:
misp("poen")

,correct,prob word,prob word g,prob of mispel,total prob
1,hello,0.267426,6.249820e-05,0.028669,1.791757e-06
3,hilly,0.004148,9.694409e-07,0.000683,6.617344e-10
0,hill,0.519337,1.213702e-04,0.000000,0.000000e+00
2,hills,0.209089,4.886449e-05,0.000000,0.000000e+00
